In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas in c:\users\meyer\documents\dom_detecter_code\dom_detecter\dom_detecter_env\lib\site-packages (2.1.0)



You should consider upgrading via the 'c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
directory = r'C:\Users\Meyer\Documents\Dom_Detecter\UNSW_csv_23-26'
all_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

In [4]:
list_of_dataframes = [pd.read_csv(os.path.join(directory, f)) for f in all_files]
merged_dataframe = pd.concat(list_of_dataframes, ignore_index=True)

In [8]:
# Read the mapping DataFrame from Excel
mapping_df = pd.read_excel(r'C:\Users\Meyer\Documents\Dom_Detecter\UNSW_csv_23-02\mapping\mapping.xlsx', engine='openpyxl')

In [9]:
df_merged = merged_dataframe

In [13]:
# Create a mapping dictionary
mapping_dict = dict(zip(mapping_df.packet_eth_src, mapping_df.target))

In [14]:
# Apply the mapping to the 'packet_eth_src' column in df_merged
df_merged['target'] = df_merged['packet_eth_src'].map(mapping_dict)

# Fill any NaN values with 'unknown'
df_merged['target'] = df_merged['target'].fillna('unknown')

In [15]:
df_merged.tail()

,idx,protocol,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,...,End_Time,Duration,Flow_Volume,Flow_Rate,Inter_Arrival_Time,Total_Fwd_Packets,Total_Bwd_Packets,Fwd_Packet_Length_Total,Bwd_Packet_Length_Total,target
2857800,573844,TCP,192.168.1.106,40767.0,52.87.241.159,443.0,66.0,ack,20392.0,64.0,...,1.474898e+09,7700.960897,2164149,281.023242,0.061734,7575,7477,1649959,514190,unknown
2857801,573845,TCP,192.168.1.106,40767.0,52.87.241.159,443.0,156.0,ack,20392.0,64.0,...,1.474898e+09,7700.960897,2164149,281.023242,0.296371,7575,7477,1649959,514190,unknown
2857802,573846,TCP,52.87.241.159,443.0,192.168.1.106,40767.0,66.0,ack,428032.0,224.0,...,1.474898e+09,7700.960897,2164149,281.023242,0.215401,7575,7477,1649959,514190,unknown
2857803,573847,TCP,192.168.1.106,40767.0,52.87.241.159,443.0,156.0,ack,20392.0,64.0,...,1.474898e+09,7700.960897,2164149,281.023242,0.800442,7575,7477,1649959,514190,unknown
2857804,573848,TCP,52.87.241.159,443.0,192.168.1.106,40767.0,66.0,ack,428032.0,224.0,...,1.474898e+09,7700.960897,2164149,281.023242,0.216231,7575,7477,1649959,514190,unknown


In [16]:
# Count the number of rows
num_rows = df_merged.shape[0]

# Print the number of rows
print(f"The DataFrame has {num_rows} rows.")

The DataFrame has 2857805 rows.


In [17]:
# Count the number of packets for each target and print it
target_counts = df_merged['target'].value_counts()
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown               2065416
amazon_echo            193340
samsung_cam            156424
belkin_motion          147695
belkin_switch          123192
smart_things            48536
netatmo_cam             45732
tp-link_cam             22305
netatmo_weather         14307
triby_speaker           13722
pixstar_photoframe      11795
hp_printer               9741
tp-link_plug             4574
withings_scale            564
nest_smoke                435
blipcare_bp_meter          27
Name: count, dtype: int64


In [18]:
# Group by device identifier and count unique flows
unique_flows_per_device = df_merged.groupby('target')['Flow'].nunique()

# Display the counts
print(unique_flows_per_device)

target
amazon_echo            9482
belkin_motion          5035
belkin_switch          3956
blipcare_bp_meter         3
hp_printer               98
nest_smoke               22
netatmo_cam            1519
netatmo_weather         491
pixstar_photoframe     2950
samsung_cam            6788
smart_things           1403
tp-link_cam             827
tp-link_plug            376
triby_speaker            75
unknown               29452
withings_scale            4
Name: Flow, dtype: int64


In [19]:
# Sum up all the unique flows
total_unique_flows = unique_flows_per_device.sum()

# Display the total count of unique flows
print(f"Total unique flows: {total_unique_flows}")

Total unique flows: 62481


In [ ]:
# Step 1: Filter out targets with less than 10,000 packets
targets_to_keep = df['target'].value_counts()
targets_to_keep = targets_to_keep[targets_to_keep >= 10000].index.tolist()
filtered_df = df[df['target'].isin(targets_to_keep)]

In [ ]:
# Step 2: Sample 10,000 packets for each target
sampled_df = pd.DataFrame()
for target in filtered_df['target'].unique():
    sampled_df = pd.concat([sampled_df, filtered_df[filtered_df['target'] == target].sample(n=10000, random_state=1)])

# Reset index of the new DataFrame
sampled_df = sampled_df.reset_index(drop=True)

In [ ]:
# Create empty DataFrames for the splits
df_9000 = pd.DataFrame()
df_1000 = pd.DataFrame()

# Stratify and split the DataFrame for each target
for target in sampled_df['target'].unique():
    target_df = sampled_df[sampled_df['target'] == target]
    
    # Sample 9000 packets for the first DataFrame
    df_train_test_split = target_df.sample(n=9000, random_state=1)
    df_train_test = pd.concat([df_9000, df_9000_split])
    
    # Drop the sampled packets and sample 1000 packets for the second DataFrame
    remaining_df = target_df.drop(df_9000_split.index)
    df_holdout_split = remaining_df.sample(n=1000, random_state=1)
    df_holdout = pd.concat([df_1000, df_1000_split])

# Reset the index for both DataFrames
df_train_test = df_9000.reset_index(drop=True)
df_holdout = df_1000.reset_index(drop=True)

In [ ]:
df_train_test.to_excel('/gdrive/MyDrive/IomT/unsw_data/flow/def_flow_data/df_test_sampled_df_packet_23_28.xlsx', index=False)

In [ ]:
df_holdout.to_excel('/gdrive/MyDrive/IomT/unsw_data/flow/def_flow_data/df_holdout_sampled_df_flow_23_28.xlsx', index=False)